In [25]:
# Imports
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import SVD, SVDpp, NMF
from surprise import SlopeOne, CoClustering
from surprise.prediction_algorithms import predictions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from helpersCoClustering import*

# Load dataset and samples into a pandas data frame
DATA_TRAIN_PATH = 'data/data_train.csv'
data_np = load_data(DATA_TRAIN_PATH)


DATA_TEST_PATH = 'data/sampleSubmission.csv'
samples = load_data(DATA_TEST_PATH)

In [27]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_np[['user_id', 'movie_id', 'rating']], reader=reader)

## Basic Co-Clustering
ID : 25590
- RMSE : 1.050
- Secondary: 0.118

In [28]:
coclus_cv = cross_validate(CoClustering(), data, cv=5, n_jobs=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0104  1.0112  1.0118  1.0115  1.0095  1.0109  0.0008  
MAE (testset)     0.8125  0.8138  0.8135  0.8137  0.8132  0.8133  0.0005  
Fit time          7.83    9.22    7.68    8.49    7.30    8.11    0.68    
Test time         1.73    1.77    1.91    1.71    1.47    1.72    0.14    


In [29]:
print('CoClustering', '\t', round(coclus_cv['test_rmse'].mean(), 4), '\t', round(coclus_cv['test_mae'].mean(), 4))

CoClustering 	 1.0109 	 0.8133


## Parameters test

In [30]:
trainset = data.build_full_trainset()
algo = CoClustering().fit(trainset)

In [31]:
samples.head()

,user_id,movie_id,rating
0,37,1,3
1,73,1,3
2,156,1,3
3,160,1,3
4,248,1,3


In [41]:
samples['rating'] = samples[['user_id', 'movie_id']] \
.apply(lambda row: algo.predict(row['user_id'], row['movie_id'])[3], axis=1)

samples.head()

,user_id,movie_id,rating
0,37,1,3.178563
1,73,1,3.200130
2,156,1,3.890346
3,160,1,3.113138
4,248,1,3.588274


In [45]:
samples['rating'] = samples['rating'].apply(round)

,user_id,movie_id,rating
0,37,1,3
1,73,1,3
2,156,1,4
3,160,1,3
4,248,1,4


In [46]:
PATHOUT = "Predictions.csv"
create_csv(PATHOUT,samples)